In [2]:
import pandas as pd

# Seus dados originais
data = {
    'empresa': ['Viagem e Turismo Ltda', 'Metalúrgica S.A.'],
    'cnpj': ['12345678000190', '98765432000101'],
    'meioContato': [
        {
            'contatos': [
                {
                    'id': 1,
                    'tipo': 'email',
                    'valor': 'email1@gmail.com',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'geral', 'status': 'ativo'},
                        {'nome': 'atendimento', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 2,
                    'tipo': 'telefone',
                    'valor': '11988886666',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'comercial', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 3,
                    'tipo': 'email',
                    'valor': 'email2@gmail.com',
                    'status': 'inativo',
                    'finalidades': [
                        {'nome': 'suporte', 'status': 'ativo'},
                        {'nome': 'marketing', 'status': 'inativo'}
                    ]
                },
                {
                    'id': 4,
                    'tipo': 'telefone',
                    'valor': '11988887777',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'suporte', 'status': 'ativo'},
                        {'nome': 'emergencia', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 5,
                    'tipo': 'email',
                    'valor': 'email3@gmail.com',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'vendas', 'status': 'ativo'}
                    ]
                }
            ]
        },
        {
            'contatos': [
                {
                    'id': 6,
                    'tipo': 'telefone',
                    'valor': '11922221111',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'geral', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 7,
                    'tipo': 'email',
                    'valor': 'email5@gmail.com',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'rh', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 8,
                    'tipo': 'telefone',
                    'valor': '11922223333',
                    'status': 'inativo',
                    'finalidades': [
                        {'nome': 'financeiro', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 9,
                    'tipo': 'email',
                    'valor': 'email6@gmail.com',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'compras', 'status': 'ativo'}
                    ]
                },
                {
                    'id': 10,
                    'tipo': 'telefone',
                    'valor': '11922224444',
                    'status': 'ativo',
                    'finalidades': [
                        {'nome': 'recepcao', 'status': 'ativo'}
                    ]
                }
            ]
        }
    ]
}

# Criando o DataFrame inicial
df = pd.DataFrame(data)

# --- Processo de Achatar o DataFrame ---

# ETAPA 1: Normalizar e Explodir os contatos da empresa
df_temp_contacts = pd.DataFrame({
    'empresa': df['empresa'],
    'cnpj': df['cnpj'],
    'contatos_list': [item['contatos'] for item in df['meioContato']]
})

df_exploded_contacts = df_temp_contacts.explode('contatos_list').reset_index(drop=True)

df_normalized_contact_details = pd.json_normalize(df_exploded_contacts['contatos_list'])

df_contacts_flat = pd.concat([
    df_exploded_contacts[['empresa', 'cnpj']],
    df_normalized_contact_details
], axis=1)

df_contacts_flat = df_contacts_flat.rename(columns={
    'id': 'contato_id',
    'tipo': 'contato_tipo',
    'valor': 'contato_valor',
    'status': 'contato_status'
})


# ETAPA 2: Normalizar e Explodir as finalidades de cada contato
df_contacts_flat['finalidades'] = df_contacts_flat['finalidades'].apply(lambda x: x if isinstance(x, list) else [])

df_exploded_finalities = df_contacts_flat.explode('finalidades').reset_index(drop=True)

df_finality_details = pd.json_normalize(df_exploded_finalities['finalidades'])

df_fully_flattened = pd.concat([
    df_exploded_finalities.drop(columns=['finalidades']),
    df_finality_details
], axis=1)

df_fully_flattened = df_fully_flattened.rename(columns={
    'nome': 'finalidade_nome',
    'status': 'finalidade_status'
})

df_fully_flattened = df_fully_flattened.dropna(subset=['finalidade_nome', 'finalidade_status']).reset_index(drop=True)

print("--- DataFrame Final TOTALMENTE ACHATADO ---")
print(df_fully_flattened)

# --- Salvar o DataFrame em um arquivo CSV ---
output_file = 'contatos_empresas_finalidades.csv'
df_fully_flattened.to_csv(output_file, index=False, encoding='utf-8')

print(f"\nDataFrame salvo com sucesso em '{output_file}'")

--- DataFrame Final TOTALMENTE ACHATADO ---
                  empresa            cnpj  contato_id contato_tipo  \
0   Viagem e Turismo Ltda  12345678000190           1        email   
1   Viagem e Turismo Ltda  12345678000190           1        email   
2   Viagem e Turismo Ltda  12345678000190           2     telefone   
3   Viagem e Turismo Ltda  12345678000190           3        email   
4   Viagem e Turismo Ltda  12345678000190           3        email   
5   Viagem e Turismo Ltda  12345678000190           4     telefone   
6   Viagem e Turismo Ltda  12345678000190           4     telefone   
7   Viagem e Turismo Ltda  12345678000190           5        email   
8        Metalúrgica S.A.  98765432000101           6     telefone   
9        Metalúrgica S.A.  98765432000101           7        email   
10       Metalúrgica S.A.  98765432000101           8     telefone   
11       Metalúrgica S.A.  98765432000101           9        email   
12       Metalúrgica S.A.  98765432000101     

| empresa,cnpj,contato_id,contato_tipo,contato_valor,contato_status,finalidade_nome,finalidade_status  |
| ---------------------------------------------------------------------------------------------------- |
| Viagem e Turismo Ltda,12345678000190,1,email,email1@gmail.com,ativo,geral,ativo                      |
| Viagem e Turismo Ltda,12345678000190,1,email,email1@gmail.com,ativo,atendimento,ativo                |
| Viagem e Turismo Ltda,12345678000190,2,telefone,11988886666,ativo,comercial,ativo                    |
| Viagem e Turismo Ltda,12345678000190,3,email,email2@gmail.com,inativo,suporte,ativo                  |
| Viagem e Turismo Ltda,12345678000190,3,email,email2@gmail.com,inativo,marketing,inativo              |
| Viagem e Turismo Ltda,12345678000190,4,telefone,11988887777,ativo,suporte,ativo                      |
| Viagem e Turismo Ltda,12345678000190,4,telefone,11988887777,ativo,emergencia,ativo                   |
| Viagem e Turismo Ltda,12345678000190,5,email,email3@gmail.com,ativo,vendas,ativo                     |
| Metalúrgica S.A.,98765432000101,6,telefone,11922221111,ativo,geral,ativo                             |
| Metalúrgica S.A.,98765432000101,7,email,email5@gmail.com,ativo,rh,ativo                              |
| Metalúrgica S.A.,98765432000101,8,telefone,11922223333,inativo,financeiro,ativo                      |
| Metalúrgica S.A.,98765432000101,9,email,email6@gmail.com,ativo,compras,ativo                         |
| Metalúrgica S.A.,98765432000101,10,telefone,11922224444,ativo,recepcao,ativo                         |